# PhysioNet/Computing in Cardiology Challenge 2020
## Classification of 12-lead ECGs
### 5. Tune Model

# Setup Noteboook

In [1]:
# 3rd party libraries
import os
import sys
import numpy as np
import tensorflow as tf

# Local Libraries
sys.path.insert(0, os.path.dirname(os.path.dirname(os.path.dirname(os.path.dirname(os.path.abspath(os.getcwd()))))))
from kardioml.models.deepecg.utils.devices.device_check import print_device_counts
from kardioml.models.deepecg.train.train import train
from kardioml.models.deepecg.model.model import Model
from kardioml import DATA_PATH, OUTPUT_PATH, LABELS_COUNT, ECG_LEADS, FS

# Configure Notebook
import warnings
warnings.filterwarnings('ignore')
%matplotlib inline
%load_ext autoreload
%autoreload 2

# 1. Hyper-Parameter Search

In [2]:
# Set data path
data_path = os.path.join(DATA_PATH, 'formatted')

# Set model name
training_run = 'hpt_11'

# Set sample length (seconds)
duration = 60

# Loop through CV folds
for param in [250, 300, 350, 400, 450, 500]:

    print('Hyper-Parameter: {}'.format(param))
    
    # Set data path
    lookup_path = os.path.join(DATA_PATH, 'deepecg')

    # Print devices
    print_device_counts()

    # Set save path for graphs, summaries, and checkpoints
    output_path = os.path.join(OUTPUT_PATH, training_run)
    os.makedirs(output_path, exist_ok=True)

    # Set model name
    model_name = 'sess_{}'.format(param)

    # Maximum number of checkpoints to keep
    max_to_keep = 1

    # Set randome states
    seed = 0 
    
    # Set sample frequency
    fs = param

    # Get training dataset dimensions
    length, channels = (int(duration * fs), len(ECG_LEADS))     

    # Number of classes
    classes = LABELS_COUNT

    # Choose network
    network_name = 'DeepECGV1'

    # Set hyper-parameter
    hyper_params = {'num_res_layers': 9, 'drop_rate': 0.3, 'kernel_size': 3, 
                    'conv_filts': 128, 'res_filts': 128, 'skip_filts': 128, 
                    'dilation': True, 'fs': fs}

    # Set network inputs
    network_parameters = {'length': length, 'channels': channels, 'classes': classes, 'seed': seed,
                          'hyper_params': hyper_params}

    # Create model
    print('Initializing Model')
    model = Model(model_name=model_name, 
                  network_name=network_name, 
                  network_parameters=network_parameters, 
                  save_path=output_path,
                  data_path=data_path,
                  lookup_path=lookup_path,
                  max_to_keep=max_to_keep)

    # Set hyper-parameters
    epochs = 60
    batch_size = 16            

    # Train model
    print('Training Start')
    train(model=model, epochs=epochs, batch_size=batch_size)
    print('Training End\n')

Hyper-Parameter: 250
Workstation has 1 CPUs.
Workstation has 2 GPUs.
Initializing Model
input | shape = (?, 15000, 12)
stem_layer_1 | shape = (?, 7500, 128)
stem_layer_2 | shape = (?, 3750, 128)
res_layer_1_skip | shape = (?, 3750, 128)
res_layer_2_skip | shape = (?, 3750, 128)
res_layer_3_skip | shape = (?, 3750, 128)
res_layer_4_skip | shape = (?, 3750, 128)
res_layer_5_skip | shape = (?, 3750, 128)
res_layer_6_skip | shape = (?, 3750, 128)
res_layer_7_skip | shape = (?, 3750, 128)
res_layer_8_skip | shape = (?, 3750, 128)
res_layer_9_skip | shape = (?, 3750, 128)
output_skip_addition | shape = (?, 3750, 128)
output_conv1 | shape = (?, 3750, 256)
output_conv2 | shape = (?, 3750, 512)
gap | shape = (?, 512)
logits | shape = (?, 9)
input | shape = (?, 15000, 12)
stem_layer_1 | shape = (?, 7500, 128)
stem_layer_2 | shape = (?, 3750, 128)
res_layer_1_skip | shape = (?, 3750, 128)
res_layer_2_skip | shape = (?, 3750, 128)
res_layer_3_skip | shape = (?, 3750, 128)
res_layer_4_skip | shape 

res_layer_5_skip | shape = (?, 7500, 128)
res_layer_6_skip | shape = (?, 7500, 128)
res_layer_7_skip | shape = (?, 7500, 128)
res_layer_8_skip | shape = (?, 7500, 128)
res_layer_9_skip | shape = (?, 7500, 128)
output_skip_addition | shape = (?, 7500, 128)
output_conv1 | shape = (?, 7500, 256)
output_conv2 | shape = (?, 7500, 512)
gap | shape = (?, 512)
logits | shape = (?, 9)
Training Start
Training End

